# NAVARCH 565 FA 24 Homework 4

### File Submission Instructions

#### Submit 5 Files:

From Problem 1:
- `Problem1.py`
- `YourNet.pth`
- `Problem1_Predictions.zip`

From Problem 2:
- `Problem2.py`
- `Problem2_Predictions.zip`

Please submit the Python files (`Problem1.py`, `Problem2.py`), two zip files of predictions (`Problem1_Predictions.zip`, `Problem2_Predictions.zip`), and one trained network file (`YourNet.pth`) that you will generate from Problem 1. Make sure to name the checkpoint file as **`YourNet.pth`**, as the autograder will look for this specific filename.

Enjoy Scene Understanding!


# Objectives

In this assignment, we will learn a fundamental technology for operating on point clouds. Compared to images, point clouds are difficult since they are an un-ordered set. While point clouds can be discretized in the form of a voxel grid to allow more traditional convolutional approaches, such a discretization is generally lossy. PointNet solved this problem by introducing a network which directly processes point clouds.

Before we start, take a few minutes to read through PointNet: https://arxiv.org/abs/1612.00593

# Grading

Grading for this assignment will consist of three main sections.

1.   Implementation of PointNet architecture. We will guide you through constructing a basic PointNet by dividing the network into small modules. When testing, we will create an instance of your class and attempt to load weights from our PointNet into your PointNet. If any layers do not match or have different names, the test will fail. We will then feed some dummy data to check the forward pass by comparing with our output.
2.   mIoU implementation. We will check that the output given some data matches our implementation.
3.   Test set performance. We will load your predictions as a zip file titled `Problem1_Predictions.zip` and assign a score based on the mIoU metric. The problem is out of 50 points with 10 points of extra credit on improving the performance of each student's **PointNet** at the end of this notebook.

# Setup Code
Before getting started we need to run some boilerplate code to set up our environment. You'll need to rerun this setup code each time you start the notebook.

First, run this cell load the [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload) extension. This allows us to edit `.py` source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [ ]:
# %load_ext autoreload
# %autoreload 2

### Google Colab Setup

Next we need to run a few commands to set up our environment on Google Colab. If you are running this notebook on a local machine you can skip this section.

Run the following cell to mount your Google Drive. Follow the link and sign in to your Google account (the same account you used to store this notebook!).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now recall the path in your Google Drive where you uploaded this notebook, fill it in below. If everything is working correctly then running the folowing cell should print the filenames from the assignment:

```
['utils.py', 'PointNet.ipynb',  'Problem1.py', 'semantic_kitti.yaml']
```

In [ ]:
import os

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a hw4/student/ folder and put all the files under PointNet folder, then 'student/PointNet'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "hw4/student/PointNet"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['semantic_kitti.yaml', 'utils.py', '__pycache__', 'Test', 'Val', 'Train', 'PointNetModule.pth', 'Problem1.py', 'YourNet.pth', 'PointNet.ipynb']


In [ ]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

In [ ]:
# Imports
import numpy as np
import torch
import yaml
from tqdm import tqdm

%matplotlib inline

Next we will check if a GPU is available

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


Once you have successfully mounted your Google Drive and located the path to this assignment, run the following cell to allow us to import from the `.py` files of this assignment. If it works correctly, it should print the message:

```
Welcome to assignment 4!
```

In [ ]:
from utils import *
from Problem1 import *
hello()

py_path = os.path.join(GOOGLE_DRIVE_PATH, 'Problem1.py')
py_edit_time = time.ctime(os.path.getmtime(py_path))
print('Student_Solution.py last edited on %s' % py_edit_time)

Welcome to assignment 4!
Student_Solution.py last edited on Thu Nov 27 21:56:59 2025


# Data Visualization

In this assignment we will be working with point clouds from Semantic KITTI again. To get an understanding of how the data looks, return to A3 where we registered semantic point clouds together. Today we will be learning to segment the point clouds ourselves with semantic labels such as car, person, or road.

Download the data from the following link and unzip it to a PointNet folder: https://curly-dataset-public.s3.us-east-2.amazonaws.com/NA565/PointNet/StudentData/PointNet.zip



If the file paths are set correctly, we should be able to visualize a point cloud below.

Notice how the data is dense close to the vehicle and sparse further away? The amount of data could cause problems for PointNet, which typically handles fewer than a hundred thousand points, so we used a voxel filter to pre-process the input. Now, the data contains at most one point per voxel with dimensions of 10 cm along each axis.

In [ ]:
# First, load the learning map for Semantic KITTI
DATA_PATH = os.path.join(GOOGLE_DRIVE_PATH)
config_file = os.path.join(DATA_PATH, "semantic_kitti.yaml")
kitti_config = yaml.safe_load(open(config_file, 'r'))
# Label map
LABELS_REMAP = kitti_config["learning_map"]

In [ ]:
%matplotlib inline

# File paths
demo_pc = os.path.join(DATA_PATH, "Train", "velodyne_ds", "000000.bin")
demo_label = os.path.join(DATA_PATH, "Train", "labels_ds", "000000.label")
# Obtain numpy arrays
demo_pc = np.fromfile(demo_pc, dtype=np.float32).reshape(-1, 4)
demo_label = np.fromfile(demo_label, dtype=np.int32).reshape(-1) & 0xFFFF
# Remap labels
label_remap = get_remap_lut(LABELS_REMAP)
demo_label = label_remap[demo_label]

# Plot
plot_cloud(demo_pc, demo_label)

Output hidden; open in https://colab.research.google.com to view.

### PyTorch Imports and Parameters

We need some imports for PyTorch to run, which are found in the code cell below. We also need to set some hyper-parameters. Don't worry about these for now, later you will get a chance to tune them.



*   `batch_size` is the number of data examples within a mini-batch.
*   `lr` is the learning rate of the network.
*   `num_epochs` is the number of epochs, or times the network will train on each individual example.





In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
batch_size = 2
lr = 0.001
num_epochs = 5

### Data Loader
Next, we will create data loaders for the training and validation set. Take a look in `Student_Solution` at class `PointLoader` for some starter code. You will need to implement the `__getitem__` function, which takes an index and returns the point cloud and label. See the above code for help fetching the items from the files or remapping the labels. Note that we need to mask the label to remove instance labels, and remap to the training labels.

In [ ]:
trainset = PointLoader(os.path.join(DATA_PATH, "Train"), label_remap,
                       device=device, data_split="Train")
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True,
                                          collate_fn=trainset.collate_fn)

valset = PointLoader(os.path.join(DATA_PATH, "Val"), label_remap,
                     device=device, data_split="Val")
val_loader = torch.utils.data.DataLoader(valset, batch_size=1,
                                          shuffle=True,
                                          collate_fn=valset.collate_fn)

In [ ]:
# Now let's try loading and visualizing a point cloud to see if it worked.
dataiter = iter(trainloader)
velos, labels = next(dataiter)
plot_cloud(velos[0].detach().cpu().numpy(), labels[0].detach().cpu().numpy())

Output hidden; open in https://colab.research.google.com to view.

# Point Net
Now that we have data available, the next step is to train a network which can process the sparse points.

**Basic Module**

PointNet contains basic modules which learn to consolidate global information about the point set through two operations:


*   Feature transformations on individual points through MLPs.
*   Global and symmetric feature aggregation through max pooling.

**Local and Global Information Aggregation**

The above operations define the basic PointNet module. However, in semantic segmentation the task is to produce point-wise predictions and the output of the module is a global set of features. The solution to this problem taken by PointNet is to concatenate each point-wise feature vector with the global feature vector.

**T-Net**

In order to ensure the output semantic labels are invariant to geometric transformations, PointNet introduces a T-Net architecture which itself functions as a PointNet module. However, T-Net predicts an affine transformation matrix which is directly applied to the coordinates of the input points.

## Encoder

First, let us create an encoder module. The encoder module contains a series of an MLP with linear layers and ReLU nonlinearities designed to encode the points within a feature space. The input to the module will be a list of channel sizes (`cs`). The first layer will contain a linear layer from `cs[0]` to `cs[1]` followed by a batch norm and ReLU. The next layer will have a linear layer from `cs[1]` to `cs[2]`, etc. If `linear_out` is not none, add a linear layer after the MLP to create un-normalized predictions as the output of the network.

For this module, you will need to understand the following functions:


1.   `torch.nn.Sequential()`
2.   `nn.Linear()`
3.   `nn.BathNorm1d()`
4.   `nn.ReLU()`


Implement `PointNetEncoder`. If it is working correctly, the following cell should return True. Note that this will only work in Google Colab, as behaviour on your local computer may be different.

In [ ]:
# Create an instance of the module
seed_torch()
net = PointNetEncoder([3, 64, 128, 1024]).to(device)

# Set all parameters to one
with torch.no_grad():
  state_dict = net.state_dict()
  for param_tensor in state_dict:
      state_dict[param_tensor] = torch.ones_like(state_dict[param_tensor])
  net.load_state_dict(state_dict)

# Dummy data
test_in = demo_pc[:1000, :3].reshape(2, -1, 3)
test_in = torch.from_numpy(test_in).to(device)

test_out = torch.sum(net(test_in)).item()

# If this works, you should see the following return True
if str(device) == "cpu":
  print(np.isclose(1071978.25, test_out))
else:
  print(np.isclose(1071978.25, test_out))

True


## Global and Local Aggregation
The PointNet encoder is useful for learning point-wise features, however there is currently no symmetric feature aggregation. In order to obtain symmetric feature aggregation, we will add a max-pooling layer after the encoder. Fill in `PointNetModule`, which uses a PointNet Encoder to learn point-wise features, followed by a max pooling to obtain global features as well as local and global feature aggregation.

The input to the module is similar, including a list of encoder channel sizes and decoder channel sizes. In the initialization, create a `PointNetEncoder` MLP using `cs_en`, and a `PointNetEncoder` MLP using `cs_dec` with linear_out set to the number of classes. In the forward pass, first pass the input through the encoder MLP. Then apply max pooling over the points dimension to obtain global features. Finally, concatenate the point-wise features with the global features and apply the decoder MLP.

Implement `PointNetModule`. If it works, the following cell should return `True`.

In [ ]:
seed_torch()
net = PointNetModule([3, 64, 128, 1024], [2048, 128, 64, 32]).to(device)

# Set all parameters to one
with torch.no_grad():
  state_dict = net.state_dict()
  for param_tensor in state_dict:
      state_dict[param_tensor] = torch.ones_like(state_dict[param_tensor])
  net.load_state_dict(state_dict)

# Dummy data
test_in = demo_pc[:1000, :3].reshape(2, -1, 3)
test_in = torch.from_numpy(test_in).to(device)

test_out = torch.sum(net(test_in)).item()

# If this works, the following should print True
if str(device) == "cpu":
  print(np.isclose(672943.1875, test_out))
else:
  print(np.isclose(672943.1875, test_out))

True


## Training

Now that we have a basic `PointNetModule`, let's try training it. Note that our network does not have a T-Net at this point so performance may be subpar. The first epoch will take a while to load all of the data, but performance will be significantly faster after the first epoch.

In [ ]:
def train_net(net, trainloader, val_loader, device, num_epochs, optimizer, criterion):
  for epoch in range(num_epochs):  # loop over the dataset multiple times
      # Train
      net.train()
      total_loss = 0
      i = 0
      loop = tqdm(trainloader)
      for data in loop:
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # zero the parameter gradients
          optimizer.zero_grad()

          # Forward pass
          outputs = net(inputs)
          B, N, C = outputs.shape
          outputs = outputs.view(-1, C)
          labels = labels.view(-1).long()

          # backward + optimize
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          loop.set_description("Training")
          total_loss += loss.item()
          i += 1
          loop.set_postfix(average_loss=total_loss/i)

      # Validate
      num_correct = 0
      num_total = 0
      net.eval()
      loop = tqdm(val_loader)
      with torch.no_grad():
        for data in loop:
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # Forward pass
            outputs = net(inputs)
            B, N, C = outputs.shape
            outputs = outputs.view(-1, C)
            labels = labels.view(-1).long()
            # Check correct
            _, predicted = torch.max(outputs, 1)
            num_correct += torch.sum(predicted == labels).item()
            num_total += predicted.shape[0]

            loop.set_description("Validation")
            loop.set_postfix(Acc=100*num_correct/num_total)

      # print statistics
      print(f'epochs: {epoch + 1} Accuracy Val: {100 * num_correct / num_total:.3f}')
  print('Finished Training')

In [ ]:
# Training will be very slow on CPU, recommend using GPU
# Be patient with the first epoch if running in Google Colab.
# The first epoch takes extra time to fetch files.
seed_torch()

lr = 0.001
num_epochs = 10

net = PointNetModule([3, 32, 64, 128], [256, 128, 64]).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)

train_net(net, trainloader, val_loader, device, num_epochs, optimizer, criterion)

# Save network
PATH = os.path.join(GOOGLE_DRIVE_PATH, 'PointNetModule.pth')
torch.save(net.state_dict(), PATH)

Validation: 100%|██████████| 150/150 [07:57<00:00,  3.18s/it, Acc=49.8]


epochs: 1 Accuracy Val: 49.814


Validation: 100%|██████████| 150/150 [00:59<00:00,  2.53it/s, Acc=41.3]


epochs: 2 Accuracy Val: 41.270


Validation: 100%|██████████| 150/150 [00:54<00:00,  2.78it/s, Acc=29.8]


...

epochs: 8 Accuracy Val: 44.059


Validation: 100%|██████████| 150/150 [00:49<00:00,  3.05it/s, Acc=53.8]


epochs: 9 Accuracy Val: 53.808


Validation: 100%|██████████| 150/150 [00:52<00:00,  2.86it/s, Acc=39]


epochs: 10 Accuracy Val: 39.031
Finished Training


## T-Net
Next, we will add the transformation network or T-Net to our PointNet module. The T-Net operates very similarly to the PointNet module, however it only operates on the global features to create a global 3x3 transformation matrix.

In the initialization function, create a `PointNetEncoder` MLP to encode and decode the transformation. Also create a `PointNetModule` for the joint encoding operation. In the forward pass:


1.   Pass the input through the transformation encoder
2.   Apply max pooling to obtain global features.
3. Pass the global features through the transformation decoder to obtain the transformation.
4. Reshape the transfomation to Bx3x3  and add an identity matrix. The identity matrix creates a possible skip connection.
5. Apply the transformation to the input points to obtain transformed points, and feed the transformed points through the `PointNetModule` joint encoder.

Fill in `PointNetFull`. If the T-Net is implemented correctly, the following cell will return `True`.

In [ ]:
# Channel sizes
cs_t_en = [3, 32, 64]
cs_t_dec = [64, 32]
cs_enc = [3, 32, 64, 128]
cs_dec = [256, 128, 64, 32]

# Create model
seed_torch()
net = PointNetFull(cs_enc, cs_dec, cs_t_en, cs_t_dec).to(device)

# Set all parameters to one
with torch.no_grad():
  state_dict = net.state_dict()
  for param_tensor in state_dict:
      state_dict[param_tensor] = torch.ones_like(state_dict[param_tensor])
  net.load_state_dict(state_dict)

test_in = demo_pc[:1000, :3].reshape(2, -1, 3)
test_in = torch.from_numpy(test_in).to(device)

test_out = torch.sum(net(test_in)).item()

# If this works, the following should print True
if str(device) == "cpu":
  print(np.isclose(668833.0, test_out))
else:
  print(np.isclose(668833.0, test_out))

True


Next, let's train the new network with the T-Net. You should see a slight difference in network performance, but do not worry if the difference is small. Later we will get a chance to improve the network further.

In [ ]:
# Training will be very slow on CPU, recommend using GPU
seed_torch()

lr = 0.001
num_epochs = 10

# Channel sizes
cs_t_en = [3, 32, 64]
cs_t_dec = [64, 32, 9]
cs_enc = [3, 32, 64, 128]
cs_dec = [256, 128, 64, 20]
net = PointNetFull(cs_enc, cs_dec, cs_t_en, cs_t_dec).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)

train_net(net, trainloader, val_loader, device, num_epochs, optimizer, criterion)

Validation: 100%|██████████| 150/150 [01:01<00:00,  2.45it/s, Acc=36.5]


epochs: 1 Accuracy Val: 36.462


Validation: 100%|██████████| 150/150 [00:58<00:00,  2.55it/s, Acc=37.8]


epochs: 2 Accuracy Val: 37.766


Validation: 100%|██████████| 150/150 [01:05<00:00,  2.28it/s, Acc=44.7]


...

epochs: 8 Accuracy Val: 35.546


Validation: 100%|██████████| 150/150 [01:00<00:00,  2.47it/s, Acc=48.1]


epochs: 9 Accuracy Val: 48.051


Validation: 100%|██████████| 150/150 [00:58<00:00,  2.55it/s, Acc=44.9]

epochs: 10 Accuracy Val: 44.862
Finished Training


## Visualization
Congratulations! You have now built your very first PointNet. Now we will visualize the predictions from your model on a point cloud from the data set. Hopefully it looks good!

In [ ]:
# Load a sample of data
dataiter = iter(trainloader)
velos, labels = next(dataiter)
with torch.no_grad():
  # forward pass
  output = net(velos)
  probabilities = torch.nn.functional.softmax(output, dim=1)
  predictions = torch.argmax(probabilities, dim=2)
  predictions_np = predictions[0, :].detach().cpu().numpy()
  velo_pc = velos[0, :, :].detach().cpu().numpy()

plot_cloud(velo_pc, predictions_np)

Output hidden; open in https://colab.research.google.com to view.

# Mean IoU
Lastly, before you attempt to build the best PointNet you can, we need to choose a different evaluation metric. While accuracy can be an important metric, it does not provide information about per-class performance or the proportion of true positives to false positives. Instead, we will evaluate with the __intersection over union (IoU)__ metric.

IoU = $\frac{\text{target} ⋂ prediction}{target ⋃ prediction}$

Implement the `IoU` function in your python file. You will be given a Pytorch tensor for the predictions and targets as input, as well as the number of classes (`C`) in total. The function should return an array of size `C` containing the IoU for each category. In the event that there are none of a class in the targets, return 1 for the class. The function should return both a per-class IoU, and mean IoU. We will use the mIoU to evaluate your network.

In [ ]:
seed_torch()
targets = torch.randint(20, (1000,), device=device)
predictions = torch.randint(20, (1000,), device=device)
test_iou, miou = IoU(targets, predictions, 20)

if str(device) == "cpu":
  print(np.isclose(0.031535230576992035, miou.item()))
else:
  print(np.isclose(0.03151371702551842, miou.item()))

True


# Your Turn!
Now it's your turn to create your own PointNet network. Try playing around with the number of layers in the encoders, the number of channels, epochs, batch sizes, and learning rate of the network. You can also add data augmentation to the data loader, such as random rotations and translations of the point cloud.

Be careful not to modify the network classes as we will be grading these. We will also be grading your network on the test set of this point cloud semantic segmentation dataset. When you are happy with the performance of your network, go to the bottom of the notebook to save the weights and generate predictions on the test set.

Note that vanilla PointNet has a difficult time with large-scale driving point clouds. In the PointNet paper, the authors propose to add another T-Net to the feature space which may help slightly increase performance.

In [ ]:
# Hyperparameters: try changing these
lr = 0.001
num_epochs = 50

# Channel sizes: try changing these
cs_t_en = [3, 16, 32]
cs_t_dec = [32, 16, 9]
cs_enc = [3, 32, 64, 128]
cs_dec = [256, 128, 64, 20]

# Data loaders
trainset = PointLoader(os.path.join(DATA_PATH, "Train"), label_remap,
                       device=device, data_split="Train")
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True,
                                          collate_fn=trainset.collate_fn)

valset = PointLoader(os.path.join(DATA_PATH, "Val"), label_remap,
                     device=device, data_split="Val")
val_loader = torch.utils.data.DataLoader(valset, batch_size=1,
                                          shuffle=True,
                                          collate_fn=valset.collate_fn)

testset = PointLoader(os.path.join(DATA_PATH, "Test"), label_remap,
                     device=device, data_split="Test")
test_loader = torch.utils.data.DataLoader(testset, batch_size=1,
                                          shuffle=False,
                                          collate_fn=testset.collate_fn)

In [ ]:
def train_net_iou(net, trainloader, val_loader, device, num_epochs, optimizer, criterion):
  for epoch in range(num_epochs):  # loop over the dataset multiple times
      # Train
      net.train()
      total_loss = 0
      i = 0
      loop = tqdm(trainloader)
      for data in loop:
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # zero the parameter gradients
          optimizer.zero_grad()

          # Forward pass
          outputs = net(inputs)
          B, N, C = outputs.shape
          outputs = outputs.view(-1, C)
          labels = labels.view(-1).long()


          # backward + optimize
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          loop.set_description("Training")
          total_loss += loss.item()
          i += 1
          loop.set_postfix(loss=total_loss / i)

      # Validate
      all_targets = []
      all_preds = []
      net.eval()
      loop = tqdm(val_loader)
      loop.set_description("Validation")
      with torch.no_grad():
        for data in loop:
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # Forward pass
            outputs = net(inputs)
            B, N, C = outputs.shape
            outputs = outputs.view(-1, C)
            labels = labels.view(-1).long()

            # Targets and predictions for iou
            _, predicted = torch.max(outputs, 1)
            all_targets.append(labels)
            all_preds.append(predicted)
      iou, miou = IoU(torch.concatenate(all_targets), torch.concatenate(all_preds), 20)

      # print statistics
      print(f'epochs: {epoch + 1} mIoU Val: {100 * miou.item():.3f}')
  print('Finished Training')

In [ ]:
# Training will be very slow on CPU, recommend using GPU
seed_torch()


net = PointNetFull(cs_enc, cs_dec, cs_t_en, cs_t_dec).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)

train_net_iou(net, trainloader, val_loader, device, num_epochs, optimizer, criterion)

Validation: 100%|██████████| 150/150 [03:02<00:00,  1.22s/it]


epochs: 1 mIoU Val: 5.246


Validation: 100%|██████████| 150/150 [00:56<00:00,  2.64it/s]


epochs: 2 mIoU Val: 7.203


Validation: 100%|██████████| 150/150 [00:51<00:00,  2.90it/s]


...

Validation: 100%|██████████| 150/150 [00:55<00:00,  2.70it/s]


epochs: 39 mIoU Val: 8.731


Validation: 100%|██████████| 150/150 [00:50<00:00,  2.94it/s]


epochs: 40 mIoU Val: 8.113


Validation:   0%|          | 0/150 [00:00<?, ?it/s]

# Submission
Congratulations on finishing the first problem of Assignment 4! Run the following code to save your weights of your network and generate predictions on the test set. For your submission, we will need **python file**, **predictions on the test set**, and **trained weights**.

Make sure to submit the predictions as a zip file named **Problem1_Predictions**.zip

In [ ]:
# Save Weights
PATH = os.path.join(GOOGLE_DRIVE_PATH, 'YourNet.pth')
torch.save(net.state_dict(), PATH)

In [ ]:
  # Generate Predictions
net.load_state_dict(torch.load(PATH))

# Test
i = 0
net.eval()
save_dir = os.path.join(DATA_PATH, "Test", "Problem1_Predictions")
if not os.path.exists(save_dir):
  os.mkdir(save_dir)
with torch.no_grad():
  for inputs, __ in iter(testset):
      # Get the inputs; data is a list of [inputs, labels]
      input = torch.from_numpy(inputs).to(device)
      input = torch.unsqueeze(input, 0)

      # Forward
      output = net(input).squeeze(0)
      _, predicted = torch.max(output, 1)

      # Save predictions
      if i % 10 == 0:
        predictions_np = predicted.detach().cpu().numpy().astype(np.int32)
        save_path = os.path.join(save_dir, str(i).zfill(6) + ".label")
        predictions_np.tofile(save_path)
      i += 1

# Next Steps
For future reading of research standards, check out the articles below:


*   PointNet++ (https://arxiv.org/abs/1706.02413) directly improves upon PointNet.
*   KPConv (https://arxiv.org/abs/1904.08889) designs point convolution which operates directly on points.
*   PointPillars (https://arxiv.org/abs/1812.05784) shows that a regular grid structure can be combined with PointNet for improved performance.

